In [7]:
import pandas as pd

df =pd.read_csv('/home/user/Desktop/jiseok/capstone/RAG/construction-safety-agent/evaluate_RAG/llm_multidb_retrieval_result.csv')
df.head()


,doctitle,question,chunk,retrieval_result,hit,selected_dbs
0,사장교 교량공사 안전보건작업 지침,사장교 교량공사 안전보건작업 지침의 목적은 무엇인가요,이지침은교량형식중주탑에경사지게설치된케이블로교량의상판을연결 지지하는형식인사장교의안전한...,['이지침은산업안전보건기준에관한규칙(이하“안전보건규칙”이라한다) 제56조 의규정(작...,False,['01_bridge']
1,교량공사의 이동식 비계공법(MSS) 안전작업 지침,이동식 비계(MSS)의 정의와 방식에 따른 종류는 무엇인가요?,(1) 이지침에서사용하는용어의정의는다음과같다.\n\n (가) “이동식비계(Mo...,['(6) 이동대차는인양하여교각브라켓상부의레일위에차륜이놓이도록설치 하고이동대차가불시...,False,"['04_scaffold', '01_bridge']"
2,강아치교(벤트공법) 안전보건작업지침,강아치교에서 아치리브(Rib)란 무엇이며 어떤 역할을 하나요?,(1) 이지침에서사용되는용어의뜻은다음과같다.\n\n (가) “강아치교”라함은주...,['(나) “주탑(Pylon)”이라함은사장교와마찬가지로주케이블의하중을기초에 전달하는...,False,['01_bridge']
3,PCT거더 교량공사 안전보건작업지침,PCT거더교의 주요 특징 5가지는 무엇인가요?,(1) PCT거더교의주요특징은다음과같다.\n\n (가) 종단및평면선형에제약을받...,"['(1) 슬라이딩패드는제작장, 횡방향가이드, 교대및교각, 추진코연결거더등 에맞는적...",False,['01_bridge']
4,현수교 교량공사 안전보건작업 지침,현수교(Suspension bridge)란 어떤 교량 형식을 말하나요?,(1) 이지침에서사용되는용어의정의는다음과같다.\n\n (가) “현수교(Susp...,['(1) 이지침에서사용되는용어의정의는다음과같다. (가) “현수교(Suspensio...,False,['01_bridge']


In [3]:
# 지석 - retriever.py


# import os
# import re
# from typing import List, Dict, Any

# from bs4 import BeautifulSoup
# from langchain.schema import Document
# from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import OpenAIEmbeddings
# from langchain_community.retrievers import BM25Retriever
# from langchain_community.cross_encoders import HuggingFaceCrossEncoder
# from langchain.retrievers.document_compressors import CrossEncoderReranker


# def get_qwen_api_embeddings():
#     embedder_model_name = "Qwen/Qwen3-Embedding-4B"
#     embedder_base_url = "http://211.47.56.71:15653/v1"
#     embedder_api_key = "token-abc123"

#     return OpenAIEmbeddings(
#         model=embedder_model_name,
#         base_url=embedder_base_url,
#         api_key=embedder_api_key,
#     )


# def _clean_text(text: str) -> str:
#     text = BeautifulSoup(text, "html.parser").get_text()
#     text = re.sub(r"\s+", " ", text)
#     return text.strip()


# class SingleDBHybridRetriever:
#     def __init__(
#         self,
#         db_dir: str,
#         top_k: int = 20,
#         alpha: float = 0.3,
#         rerank_top_n: int = 5,
#         reranker_model: str = "BAAI/bge-reranker-v2-m3"
#     ):
#         self.db_dir = db_dir
#         self.top_k = top_k
#         self.alpha = alpha
#         self.rerank_top_n = rerank_top_n
#         self.reranker_model = reranker_model

#         print(f"📂 HybridRetriever 초기화: {db_dir}")

#         # 1) load FAISS
#         self.embeddings = get_qwen_api_embeddings()
#         self.vector_db = FAISS.load_local(
#             db_dir, self.embeddings, allow_dangerous_deserialization=True
#         )

#         # BM25를 위한 전체 문서 로드
#         self.all_docs = list(self.vector_db.docstore._dict.values())

#         # 🔥 Reranker를 여기서 한 번만 초기화
#         print(f"🔄 Reranker 모델 로딩: {reranker_model}")
#         self.reranker = HuggingFaceCrossEncoder(model_name=reranker_model)
#         self.compressor = CrossEncoderReranker(model=self.reranker, top_n=rerank_top_n)
#         print(f"✅ Reranker 로딩 완료")

#     def _hybrid_merge(self, dense_results, sparse_results):
#         dense_dict = {hash(doc.page_content): score for doc, score in dense_results}
#         sparse_dict = {hash(doc.page_content): i for i, doc in enumerate(sparse_results)}

#         merged = []
#         for doc, ds in dense_results:
#             h = hash(doc.page_content)
#             sr = sparse_dict.get(h, len(sparse_results))
#             score = self.alpha * ds + (1 - self.alpha) * (1 - sr / len(sparse_results))
#             merged.append((doc, score))

#         for i, doc in enumerate(sparse_results):
#             h = hash(doc.page_content)
#             if h not in dense_dict:
#                 score = (1 - self.alpha) * (1 - i / len(sparse_results))
#                 merged.append((doc, score))

#         merged.sort(key=lambda x: x[1], reverse=True)
#         return [doc for doc, _ in merged]

#     def retrieve(self, query: str) -> List[Document]:
#         print(f"\n🔍 [HybridRetriever] Query: {query}")

#         # 1) Dense(AI semantic)
#         dense = self.vector_db.similarity_search_with_score(query, k=self.top_k)

#         # 2) Sparse(keyword)
#         sparse_retriever = BM25Retriever.from_documents(self.all_docs)
#         sparse_retriever.k = self.top_k * 4
#         sparse = sparse_retriever.get_relevant_documents(query)

#         # 3) Hybrid merge
#         hybrid_docs = self._hybrid_merge(dense, sparse)

#         # 4) Rerank - 이미 초기화된 compressor 사용
#         reranked = self.compressor.compress_documents(hybrid_docs, query)

#         # 5) Clean & return top_k
#         final_docs = []
#         for d in reranked[: self.top_k]:
#             d.page_content = _clean_text(d.page_content)
#             final_docs.append(d)

#         print(f"📊 최종 반환 문서: {len(final_docs)}개")
#         return final_docs

In [8]:
import pandas as pd

df = pd.read_csv('/home/user/Desktop/jiseok/capstone/RAG/construction-safety-agent/evaluate_RAG/llm_multidb_retrieval_result.csv')
df.head()

,doctitle,question,chunk,retrieval_result,hit,selected_dbs
0,사장교 교량공사 안전보건작업 지침,사장교 교량공사 안전보건작업 지침의 목적은 무엇인가요,이지침은교량형식중주탑에경사지게설치된케이블로교량의상판을연결 지지하는형식인사장교의안전한...,['이지침은산업안전보건기준에관한규칙(이하“안전보건규칙”이라한다) 제56조 의규정(작...,False,['01_bridge']
1,교량공사의 이동식 비계공법(MSS) 안전작업 지침,이동식 비계(MSS)의 정의와 방식에 따른 종류는 무엇인가요?,(1) 이지침에서사용하는용어의정의는다음과같다.\n\n (가) “이동식비계(Mo...,['(6) 이동대차는인양하여교각브라켓상부의레일위에차륜이놓이도록설치 하고이동대차가불시...,False,"['04_scaffold', '01_bridge']"
2,강아치교(벤트공법) 안전보건작업지침,강아치교에서 아치리브(Rib)란 무엇이며 어떤 역할을 하나요?,(1) 이지침에서사용되는용어의뜻은다음과같다.\n\n (가) “강아치교”라함은주...,['(나) “주탑(Pylon)”이라함은사장교와마찬가지로주케이블의하중을기초에 전달하는...,False,['01_bridge']
3,PCT거더 교량공사 안전보건작업지침,PCT거더교의 주요 특징 5가지는 무엇인가요?,(1) PCT거더교의주요특징은다음과같다.\n\n (가) 종단및평면선형에제약을받...,"['(1) 슬라이딩패드는제작장, 횡방향가이드, 교대및교각, 추진코연결거더등 에맞는적...",False,['01_bridge']
4,현수교 교량공사 안전보건작업 지침,현수교(Suspension bridge)란 어떤 교량 형식을 말하나요?,(1) 이지침에서사용되는용어의정의는다음과같다.\n\n (가) “현수교(Susp...,['(1) 이지침에서사용되는용어의정의는다음과같다. (가) “현수교(Suspensio...,False,['01_bridge']


In [ ]:
# # question
# # contexts (Retriever가 실제로 Top-k로 가져온 chunk들)
# # ground_truth   사람이 만든 기준 정답)

# import ast


# df1 = df[['question', 'retrieval_result', 'chunk']].rename(
#     columns={
#         'retrieval_result': 'contexts',
#         'chunk': 'ground_truth'
#     }
# )

# df1['contexts'] = df1['contexts'].apply(ast.literal_eval)
# df1.head()

,question,contexts,ground_truth
0,사장교 교량공사 안전보건작업 지침의 목적은 무엇인가요,[이지침은산업안전보건기준에관한규칙(이하“안전보건규칙”이라한다) 제56조 의규정(작업...,이지침은교량형식중주탑에경사지게설치된케이블로교량의상판을연결 지지하는형식인사장교의안전한...
1,이동식 비계(MSS)의 정의와 방식에 따른 종류는 무엇인가요?,[(6) 이동대차는인양하여교각브라켓상부의레일위에차륜이놓이도록설치 하고이동대차가불시에...,(1) 이지침에서사용하는용어의정의는다음과같다.\n\n (가) “이동식비계(Mo...
2,강아치교에서 아치리브(Rib)란 무엇이며 어떤 역할을 하나요?,[(나) “주탑(Pylon)”이라함은사장교와마찬가지로주케이블의하중을기초에 전달하는역...,(1) 이지침에서사용되는용어의뜻은다음과같다.\n\n (가) “강아치교”라함은주...
3,PCT거더교의 주요 특징 5가지는 무엇인가요?,"[(1) 슬라이딩패드는제작장, 횡방향가이드, 교대및교각, 추진코연결거더등 에맞는적정...",(1) PCT거더교의주요특징은다음과같다.\n\n (가) 종단및평면선형에제약을받...
4,현수교(Suspension bridge)란 어떤 교량 형식을 말하나요?,[(1) 이지침에서사용되는용어의정의는다음과같다. (가) “현수교(Suspension...,(1) 이지침에서사용되는용어의정의는다음과같다.\n\n (가) “현수교(Susp...


In [5]:
!pip install ragas

  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached nodeenv-1.9.1-py2.py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s  0:00:00
Using cached nodeenv-1.9.1-py2.py3-none-any.whl (22 kB)
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import pandas as pd
import ast
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall
from langchain_openai import ChatOpenAI


OPENAI_API_KEY = ""

# LLM 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY,
    temperature=0
)

# /home/tmdrb/construction-safety-reporteragent/llm_multidb_retrieval_result.csv' ->  해당 csv 기준 
def load_and_prepare_data(csv_path):
    """
    CSV 파일을 로드하고 RAGAS 평가를 위한 형식으로 변환
    
    Args:
        csv_path: CSV 파일 경로
    
    Returns:
        전처리된 데이터프레임
    """
    # CSV 로드
    df = pd.read_csv(csv_path)
    
    # 컬럼명 변경 (필요한 경우)
    df1 = df[['question', 'retrieval_result', 'chunk']].rename(
        columns={
            'retrieval_result': 'contexts',
            'chunk': 'ground_truth'
        }
    )
    
    # contexts를 리스트로 변환 (문자열로 저장되어 있는 경우)
    if isinstance(df1['contexts'].iloc[0], str):
        df1['contexts'] = df1['contexts'].apply(ast.literal_eval)
    
    return df1


def prepare_ragas_dataset(df):
    """
    데이터프레임을 RAGAS Dataset 형식으로 변환
    """
    dataset_dict = {
        'question': df['question'].tolist(),
        'contexts': df['contexts'].tolist(),
        'ground_truth': df['ground_truth'].tolist()
    }
    return Dataset.from_dict(dataset_dict)


# 3. RAGAS 평가 실행
def evaluate_retriever(df, llm_model):
    """
    Retriever 성능을 평가하고 각 행별로 점수를 반환
    
    Args:
        df: 평가할 데이터프레임 (question, contexts, ground_truth 컬럼 필요)
        llm_model: RAGAS 평가에 사용할 LLM 모델
    
    Returns:
        원본 데이터프레임에 context_precision, context_recall 컬럼이 추가된 데이터프레임
    """
    
    # Dataset 생성
    ragas_dataset = prepare_ragas_dataset(df)
    
    # RAGAS 평가 실행
    print("RAGAS 평가를 시작합니다...")
    print(f"평가 데이터 수: {len(ragas_dataset)}")
    
    results = evaluate(
        dataset=ragas_dataset,
        metrics=[
            context_precision,  # 검색된 문서의 정밀도 (쓸모있는 문서 비율)
            context_recall      # 검색된 문서의 재현율 (필요한 정보 포함 여부)
        ],
        llm=llm_model
    )
    
    print("평가가 완료되었습니다!")
    
    # 결과를 데이터프레임으로 변환
    results_df = results.to_pandas()
    
    # 원본 데이터프레임에 점수 추가
    df_with_scores = df.copy()
    df_with_scores['context_precision'] = results_df['context_precision']
    df_with_scores['context_recall'] = results_df['context_recall']
    
    return df_with_scores, results


def main(csv_path, output_path=None):
    """
    CSV 파일을 로드하고 RAGAS 평가를 수행하는 메인 함수
    
    Args:
        csv_path: 입력 CSV 파일 경로
        output_path: 결과 저장 경로 (None이면 저장하지 않음)
    
    Returns:
        평가가 완료된 데이터프레임
    """
    
    print("=" * 60)
    print("데이터 로드 중...")
    print("=" * 60)
    
    # 데이터 로드 및 전처리
    df1 = load_and_prepare_data(csv_path)
    
    print(f"총 {len(df1)}개의 데이터가 로드되었습니다.")
    print("\n원본 데이터 샘플:")
    print(df1.head())
    
    # 평가 실행
    df_evaluated, eval_results = evaluate_retriever(df1, llm)
    
    print("\n" + "=" * 60)
    print("평가 결과:")
    print("=" * 60)
    print(df_evaluated[['question', 'context_precision', 'context_recall']].head())
    
    # 전체 평균 점수
    print("\n" + "=" * 60)
    print("전체 평균 점수:")
    print("=" * 60)
    print(f"Context Precision 평균: {df_evaluated['context_precision'].mean():.4f}")
    print(f"Context Recall 평균: {df_evaluated['context_recall'].mean():.4f}")
    
    # 낮은 점수 케이스 확인
    print("\n" + "=" * 60)
    print("Context Precision이 낮은 상위 5개:")
    print("=" * 60)
    low_precision = df_evaluated.nsmallest(5, 'context_precision')[
        ['question', 'context_precision', 'context_recall']
    ]
    print(low_precision)
    
    print("\n" + "=" * 60)
    print("Context Recall이 낮은 상위 5개:")
    print("=" * 60)
    low_recall = df_evaluated.nsmallest(5, 'context_recall')[
        ['question', 'context_precision', 'context_recall']
    ]
    print(low_recall)
    
    if output_path:
        df_evaluated.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"\n결과가 '{output_path}'에 저장되었습니다.")
    
    return df_evaluated


if __name__ == "__main__":

    CSV_PATH = "/home/user/Desktop/jiseok/capstone/RAG/construction-safety-agent/evaluate_RAG/llm_multidb_retrieval_result.csv"  
    OUTPUT_PATH = "RAGAS_retriever_evaluation_results(multi).csv"  
    
    df_result = main(CSV_PATH, OUTPUT_PATH)

데이터 로드 중...
총 99개의 데이터가 로드되었습니다.

원본 데이터 샘플:
                                  question  \
0            사장교 교량공사 안전보건작업 지침의 목적은 무엇인가요   
1       이동식 비계(MSS)의 정의와 방식에 따른 종류는 무엇인가요?   
2       강아치교에서 아치리브(Rib)란 무엇이며 어떤 역할을 하나요?   
3                PCT거더교의 주요 특징 5가지는 무엇인가요?   
4  현수교(Suspension bridge)란 어떤 교량 형식을 말하나요?   

                                            contexts  \
0  [이지침은산업안전보건기준에관한규칙(이하“안전보건규칙”이라한다) 제56조 의규정(작업...   
1  [(6) 이동대차는인양하여교각브라켓상부의레일위에차륜이놓이도록설치 하고이동대차가불시에...   
2  [(나) “주탑(Pylon)”이라함은사장교와마찬가지로주케이블의하중을기초에 전달하는역...   
3  [(1) 슬라이딩패드는제작장, 횡방향가이드, 교대및교각, 추진코연결거더등 에맞는적정...   
4  [(1) 이지침에서사용되는용어의정의는다음과같다. (가) “현수교(Suspension...   

                                        ground_truth  
0  이지침은교량형식중주탑에경사지게설치된케이블로교량의상판을연결 지지하는형식인사장교의안전한...  
1  (1) 이지침에서사용하는용어의정의는다음과같다.\n\n    (가) “이동식비계(Mo...  
2  (1) 이지침에서사용되는용어의뜻은다음과같다.\n\n    (가) “강아치교”라함은주...  
3  (1) PCT거더교의주요특징은다음과같다.\n\n    (가) 종단및평면선형에제약을받...  
4  (1) 이지침에서사용되는용어의정의는다음과같다.\n\n    (가) “현수교(Susp...  
RAGAS 평가를 시

Evaluating:   0%|          | 0/198 [00:00<?, ?it/s]

Exception raised in Job[106]: TimeoutError()
Exception raised in Job[138]: TimeoutError()
Exception raised in Job[152]: TimeoutError()


평가가 완료되었습니다!

평가 결과:
                                  question  context_precision  context_recall
0            사장교 교량공사 안전보건작업 지침의 목적은 무엇인가요           1.000000             1.0
1       이동식 비계(MSS)의 정의와 방식에 따른 종류는 무엇인가요?           1.000000             1.0
2       강아치교에서 아치리브(Rib)란 무엇이며 어떤 역할을 하나요?           1.000000             1.0
3                PCT거더교의 주요 특징 5가지는 무엇인가요?           0.988889             1.0
4  현수교(Suspension bridge)란 어떤 교량 형식을 말하나요?           1.000000             1.0

전체 평균 점수:
Context Precision 평균: 0.9914
Context Recall 평균: 0.8287

Context Precision이 낮은 상위 5개:
                                             question  context_precision  \
34  블록식 보강토 옹벽 시공 완료 후 옹벽 높이에 대한 허용 기울기(수직도) 기준은 얼...           0.645370   
63  타워크레인 설치, 조립, 해체 작업을 하는 경우 작업 계획서에 반드시 포함되어야 하...           0.841226   
72  미장공사 완료 후, 가시설 해체 작업 시 2m 이상 고소부위에서 추락 방지를 위해 ...           0.878263   
64  이동식 크레인 인양 작업 중, 악천후로 인해 작업을 중지해야 하는 풍속 기준은 초당...           0.884354   
32  사질지반(점토질 미포함) 굴착 시 안전을 확보하기 위해 